This program will grab the current active watches and warnings from the NWS API service and plot them on the grids using the python-awips coloring package for NWS watches and warnings.

Note: In the awips-python package, there is currently no coloring set for special weather statements and a couple others. Those colors will be prepended as this program is updated. Currently, special weather statements are prepended to be shaded in 'moccasin'.

In [ ]:
#Import the necessary packages
from noaa_sdk import noaa
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import ShapelyFeature
from shapely.geometry import Polygon,MultiPolygon
from awips.tables import vtec
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import datetime

We import cartopy packages for the plotting as well as the noaa_sdk which serves as our loading point for the NWS API. This package is maintained by another source, so things could potentially break, but this program will be ammended as that happens.

Now let's define the functions that we need for grabbing our colors and creating the map. We are going to start with an overall map of the United States. Hopefully we will allow for zoomability into smaller areas that will allow for legends to be displayed telling what type of alert is occuring.

In [ ]:
#Function we will need to grab the warning colors
def warning_color(phensig):
    if(phensig == 'Special Weather Statement' or phensig == "Marine Weather Statement"):
        color = 'moccasin'
    elif(phensig == "Rip Current Statement"):
        color = "aqua"
    else:
        for val in vtec:
            event_val = vtec[val]['hdln']
            if(event_val == phensig):
                color = vtec[val]['color']
    return color


#Function to create the initial map
def make_map(proj=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize = (16,12), subplot_kw=dict(projection=proj))
    ax.set_extent([-130, -60, 24, 50.5])
    gl = ax.gridlines(draw_labels=True, color='#e7e7e7')
    gl.xlabels_top = gl.ylabels_right = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
#Open the NWS API in python to get the active alerts
n = noaa.NOAA()
alerts = n.active_alerts()

#Make our initial map
fig, ax = make_map()

#Add in features to our map such as states, lakes and oceans
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAKES.with_scale('50m'))

#Create the county shapefile to be added on top of the warning map
reader = shpreader.Reader('countyl010g.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

In [2]:
#Now loop through each of the individual alerts in the overall alerts. Need to take the 'features' class to
#access the 'events' and 'geometry' properties.
for alert in alerts['features']:
    # Now use a try and except statement to test if there is an alert. If no alert, will go to next one
    # If there is an alert, we will plot the shapefile of the alert on the map
    try:
        # Get the event name
        event = alert['properties']['event']
        # Now get the coordinates listed for the alert. Sometimes it is a MultiPolygon object, so
        # we use a loop to grab each object in the 'coordinates' property to be able to plot everything.
        for zz in alert['geometry']['coordinates']:
            # Sometimes coordinates will return None, if so, skip over it.
            try:
                # Get the coordinates
                poly_coords = zz

                # get the x and y coordinates into two separate lists
                xcoor, ycoor = map(list, zip(*poly_coords))

                # Now place them into tuples for each coordinate pair and put the tuples into a list
                coords = list(zip(xcoor, ycoor))

                # Create a shapely.Polygon object from the coordinates
                polygon = Polygon(coords)

                # Call the warning_color function to get the color to plot
                wcolor = warning_color(event)

                # Now add the feature to the axis. We use [polygon] as the ShapelyFeature takes a list of objects
                shape_feature = ShapelyFeature([polygon], ccrs.PlateCarree(),
                                               facecolor=wcolor, edgecolor=wcolor)
                ax.add_feature(shape_feature)

            except:
                pass
    except:
        # Sometimes, the coordinates return None due to NWS not listing them on the API for the alert.
        # In this case, we grab the affected zone(s) and get their coordinates to plot the alert for
        try:
            # Grab each of the affected zones (note there can be just a single affected zone)
            for zi in alert['properties']['affectedZones']:

                # Grabbing the zone leaves a http link to the zone itself, so we need to parse it
                # We can parse the link at each '/' character and put it all into a list
                zone_link = str(zi)
                zone_link = zone_link.split('/')

                # Then we grab the last two elements of the list. The last element is the zone and the
                # second to last element is the type (forecast, area, etc).
                zone = zone_link[-1]
                type = zone_link[-2]

                # Now use the NWS API call for zones. Requires a type and a zone to be called.
                zone_info = n.zones(type, zone)

                # Use another try and except block to check for the coordinates.
                # Sometimes the coordinates are contained in 'geometry''geometries', so we need to check.
                try:
                    # Same as above. If it is a MultiPolygon, we loop through each set of coordinates.
                    # Will still work if only a Polygon
                    for zz in zone_info['geometry']['coordinates']:
                        # PC represents the coordinates inside the MultiPolygon coordinates variable (zz)
                        for pc in zz:
                            # Now try and plot, if coordinates return None, skip them.
                            try:
                                # Grab the set of coordinates
                                poly_coords = pc

                                # get the x and y coordinates into two separate lists
                                xcoor, ycoor = map(list, zip(*poly_coords))

                                # Combine the two lists into tuples of coordinates and put the tuples into
                                # a new list
                                coords = list(zip(xcoor, ycoor))

                                # Create a shapely.Polygon from the coordinate tuples
                                polygon = Polygon(coords)

                                # Call the warning_color function and get the color for the event
                                wcolor = warning_color(event)

                                # Now add the feature to the axis. We use [polygon] as the ShapelyFeature takes a list of objects
                                shape_feature = ShapelyFeature([polygon], ccrs.PlateCarree(),
                                                               facecolor=wcolor, edgecolor=wcolor)
                                ax.add_feature(shape_feature)
                            except:
                                pass
                except:
                    # Same as above for when the coordinates are in 'geometry''geometries'
                    for zg in zone_info['geometry']['geometries']:
                        try:
                            # Have to grab the coordinates out of the 'geometry''geometries' list
                            for zgg in zg['coordinates']:
                                try:

                                    for pc in zgg:
                                        # Now try and plot, if coordinates return None, skip them.
                                        try:
                                            # Grab the set of coordinates
                                            poly_coords = pc

                                            # get the x and y coordinates into two separate lists
                                            xcoor, ycoor = map(list, zip(*poly_coords))

                                            # Combine the two lists into tuples of coordinates and put the tuples into
                                            # a new list
                                            coords = list(zip(xcoor, ycoor))

                                            # Create a shapely.Polygon from the coordinate tuples
                                            polygon = Polygon(coords)

                                            # Call the warning_color function and get the color for the event
                                            wcolor = warning_color(event)

                                            # Now add the feature to the axis. We use [polygon] as the ShapelyFeature takes a list of objects
                                            shape_feature = ShapelyFeature([polygon], ccrs.PlateCarree(),
                                                                           facecolor=wcolor, edgecolor=wcolor)
                                            ax.add_feature(shape_feature)
                                        except:
                                            pass
                                except:
                                    pass
                        except:
                            pass
        except:
            pass





#Add in the coastline and counties to the map. This allows us to overlay them over the shapefiles we colored in
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(COUNTIES, facecolor='none', edgecolor='black')

#Add the plot title
ax.set_title("Current Advisories Valid: " + datetime.datetime.now().strftime("%I:%M %p"), pad =75)

#Save the figure
plt.savefig('testing.png', bbox_inches='tight')

NameError: name 'alerts' is not defined

In [ ]:
#T